Problem 1: Sales rep performance analysis - Management wants to evaluate sales representative performance to determine bonus eligibility and identify coaching opportunities. Create an analysis that shows each sales representative's total sales revenue, number of active customers, and average order value for customers under their management. Include only sales reps who have at least one customer with orders and rank them by total sales revenue.

```postgresql

```sql
SELECT e.employeeNumber, e.firstName || ' ' || e.lastName AS salesRepName,
	COUNT(DISTINCT c.customerNumber) AS activeCustomers,
	COUNT(DISTINCT o.orderNumber) AS totalOrders,
	ROUND(SUM(od.quantityOrdered * od.priceEach)::numeric, 2) AS totalSalesRevenue,
	ROUND(AVG(od.quantityOrdered * od.priceEach)::numeric, 2) AS avgOrderValue
FROM employees e
	INNER JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
	INNER JOIN orders o ON c.customerNumber = o.customerNumber
	INNER JOIN orderdetails od ON o.orderNumber = od.orderNumber
WHERE e.jobTitle LIKE '%Sales Rep%'
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING SUM(od.quantityOrdered * od.priceEach) > 0
ORDER BY totalSalesRevenue DESC;


![Sales Rep Answers](sale_rep_answers.png)

Problem 2: Profitability and Inventory Optimization - The procurement team wants to optimize inventory investments by understanding which products generate the highest profit margins and how current stock levels align with sales velocity. Calculate profit margin, total profit, and inventory turnover metrics for each product to guide purchasing decisions.

```sql
SELECT pl.productLine, pl.textDescription,
	COUNT(DISTINCT p.productCode) AS productCount,
	ROUND(AVG(p.MSRP - p.buyPrice)::numeric, 2) AS avgProfitPerUnit,
	ROUND(AVG((p.MSRP - p.buyPrice) / p.buyPrice * 100)::numeric, 2) AS avgProfitMarginPct,
	ROUND(SUM(od.quantityOrdered * (od.priceEach - p.buyPrice))::numeric,2) AS totalProfit,
	ROUND(AVG(p.quantityInStock)::numeric, 0) AS avgInventoryLevel,
	ROUND((SUM(od.quantityOrdered)::numeric / AVG(p.quantityInStock)::numeric),2) AS inventoryTurnoverRatio
FROM productlines pl
	INNER JOIN products p ON pl.productLine = p.productLine
	INNER JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY pl.productLine, pl.textDescription
ORDER BY totalProfit DESC;

![Profitability and Inventory](profitability_inventory_answers.png)

Problem 3: Customer Retention Analysis - The marketing team needs to identify at-risk customers for a retention campaign. Find customers who have made purchases but haven't placed an order in the last 6 months of available data in the database. Include their total historical order value and last order date to help prioritize outreach efforts. Only include customers who have spent more than $5000 historically.

```sql
SELECT c.customerNumber, c.customerName, c.country,
	COUNT(o.orderNumber) AS totalOrders,
	ROUND(SUM(od.quantityOrdered * od.priceEach)::numeric, 2) AS totalSpent,
	MAX(o.orderDate) AS lastOrderDate,
	CURRENT_DATE - MAX(o.orderDate::date) AS daysSinceLastOrder
FROM customers c
	INNER JOIN orders o ON c.customerNumber = o.customerNumber
	INNER JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY c.customerNumber, c.customerName, c.country
HAVING SUM(od.quantityOrdered * od.priceEach) > 5000
	AND MAX(o.orderDate::date) < CURRENT_DATE - INTERVAL '180 days'
ORDER BY totalSpent DESC, daysSinceLastOrder DESC;

![Customer Retention](customer_retention_ansers.png)